In [1]:
from surprise import SVD, SVDpp, Reader, Dataset, accuracy, NMF
import gzip
import csv
import pandas as pd
from surprise.model_selection import train_test_split

In [2]:
path="trainInteractions.csv.gz"

In [3]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    for l in c:
        d = dict(zip(header,l))
        yield d

In [4]:
dataset = list(readCSV(path))

In [5]:
df = pd.DataFrame(dataset)

In [6]:
df=df[['user_id','recipe_id','rating']]

In [7]:
reader = Reader(line_format='user item rating', sep=',', skip_lines=1)

In [8]:
data = Dataset.load_from_df(df, reader=reader)

In [9]:
trainset, testset = train_test_split(data, test_size=0.2)
# trainset=data.build_full_trainset()

In [10]:
for i in range(100):
    for j in range(3):
        model = SVD(n_factors=2+j,n_epochs=20,reg_all=0.0001*i,verbose=False)
        model.fit(trainset)
        predictions = model.test(testset)
        sse = 0
        for p in predictions:
            sse += (p.r_ui - p.est)**2
        print(i)
        print(j)
        print(sse / len(predictions))

0
0
0.8220447482295639
0
1
0.8220978439150602
0
2
0.8224600589306241
1
0
0.8215165890155739
1
1
0.8216845822503374
1
2
0.8224665516716099
2
0
0.8213924015053667
2
1
0.8219466077293589
2
2
0.8226070973337485
3
0
0.8212726662583655
3
1
0.8217207997386206
3
2
0.8227506275292558
4
0
0.8214459685032192
4
1
0.8216307164694924
4
2
0.8222945398454344
5
0
0.8217080601280773
5
1
0.8214719065576879
5
2
0.8228415728881439
6
0
0.8216440756200803
6
1
0.8222188908213856
6
2
0.822516965604095
7
0
0.8220352622619239
7
1
0.8221934689403337
7
2
0.8217365908140455
8
0
0.8210047823768561
8
1
0.8214749267900581
8
2
0.8223546674954227
9
0
0.8219848985001181
9
1
0.8215306855402659
9
2
0.8229751451456077
10
0
0.8219351213375723
10
1
0.821923920426868
10
2
0.8228055896447152
11
0
0.8213486338615891
11
1
0.8219825188575984
11
2
0.8229219558284607
12
0
0.8209732089583832
12
1
0.8220661167105212
12
2
0.8220211982288305
13
0
0.8214781276148765
13
1
0.8217548372450899
13
2
0.8220354733421197
14
0
0.8210265058334233


KeyboardInterrupt: 

In [42]:
# model = SVD(n_factors=2,n_epochs=60,reg_all=0.0020,reg_pu=0.002,reg_qi=0.002, verbose=True)
# model = SVD(n_factors=2,n_epochs=500,reg_all=0.019, verbose=True)
model = SVDpp(n_factors=2,n_epochs=10,reg_all=0.0019,verbose=True)

In [43]:
model.fit(trainset)

 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9


In [44]:
predictions = model.test(testset)

In [45]:
sse = 0
for p in predictions:
    sse += (p.r_ui - p.est)**2

print(sse / len(predictions))

0.8267111082064023


In [14]:
model.bu

array([-0.27611775, -0.15483926, -0.69070331, ...,  0.04924807,
        0.03861461,  0.04303734])

In [15]:
predictions = open("predictions_Rated.txt", 'w')
for l in open("stub_Rated.txt"):
    if l.startswith("user_id"):
        
      #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    predictions.write(u + '-' + i + ',' + str(model.predict(u,i).est) + '\n')
#     if u in userBiases and i in itemBiases:
#         predictions.write(u + '-' + i + ',' + str(prediction_(u, i)) + '\n')
#     elif i in itemBiases:
#         predictions.write(u + '-' + i + ',' + str(itemAverage[i]) + '\n')
#     elif u in userBiases:
#         predictions.write(u + '-' + i + ',' + str(userAverage[u]) + '\n')
#     else:
#         predictions.write(u + '-' + i + ',' + str(globalAverage) + '\n')

predictions.close()